In [240]:
'''Felix Andersson, Janine de Vries, DV2626'''

import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, accuracy_score, mean_squared_error
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
import ast
from IPython.display import clear_output
import random

In [ ]:
"""
Loads exercise data from CSV files, extracts important columns related to exercises and muscle groups.
Preparing a list of columns to be encoded and their corresponding ID columns.
"""
original_training_data = pd.read_csv("training_knn.csv")
exercise_data = pd.read_csv('gym.csv', delimiter=';')
important_data = exercise_data[['Exercise Name', 'Main_muscle', 'Mechanics', 'Target_Muscles', 'Synergist_Muscles', 'Stabilizer_Muscles', 'Antagonist_Muscles', 'Dynamic_Stabilizer_Muscles', 'Difficulty (1-5)', 'Secondary Muscles']].copy()
encode_columns = ['Main_muscle', 'Mechanics','Target_Muscles', 'Synergist_Muscles', 'Stabilizer_Muscles', 'Antagonist_Muscles', 'Dynamic_Stabilizer_Muscles', 'Secondary Muscles']
ID_columns = ["ID_Main_muscle", "ID_Mechanics", "ID_Target_Muscles", "ID_Synergist_Muscles", "ID_Stabilizer_Muscles", "ID_Antagonist_Muscles", "ID_Dynamic_Stabilizer_Muscles", "ID_Secondary Muscles"]

important_data.head()

In [242]:
def encoded_data(specific_muscle):
    new_data = important_data.copy()
    target_muscle_mapping = []  # List to store muscle names at correct indices
    
    for column in encode_columns:
        new_data.loc[:, column] = new_data[column].fillna("")
        new_data.loc[:, column] = new_data[column].apply(lambda x: x.split(", ") if x else [])
        
        # Multi-label binarization
        mlb = MultiLabelBinarizer()
        binary_matrix = mlb.fit_transform(new_data[column])
        
        # Save muscle names in a list if the column is "Target_Muscles"
        if column == "Target_Muscles":
            target_muscle_mapping = list(mlb.classes_)        
        new_data.loc[:, f"ID_{column}"] = binary_matrix.tolist()
        new_data = new_data.drop(columns=[column])
    finished_data = pd.DataFrame()
    for i in range(len(target_muscle_mapping)):
        muscle_index=[]
        if target_muscle_mapping[i].replace(",","") == specific_muscle:
            muscle_index.append(i)
        for index in muscle_index:
            temporary_dataset = new_data[new_data["ID_Target_Muscles"].apply(lambda x: x[index] == 1)]
            finished_data = pd.concat([finished_data, temporary_dataset], ignore_index=True)
    return finished_data


In [243]:
def adjust_score(predicted_scores, user_fitness_level, exercise_difficulty, max_difference=4):
    """
    Adjust scores based on the user's fitness level and exercise difficulty.
    """
    adjusted_scores = []
    for pred, difficulty in zip(predicted_scores, exercise_difficulty):
        factor = 1 - (abs(user_fitness_level - difficulty) / max_difference) ** 2
        factor = max(0.5, factor)
        adjusted_scores.append(pred * factor)
    return adjusted_scores

def knn(predict_data, user_fitness_level,The_Exercises):
    """
    Perform KNN classification and adjust the predicted scores.
    """
    training_data = original_training_data.copy()
    for col in ID_columns:
        def safe_literal_eval(value):
            try:
                # Ensure the value is parsed correctly
                if isinstance(value, str):
                    return np.array(ast.literal_eval(value))
                elif isinstance(value, list):
                    return np.array(value)
                else:
                    return np.array([])  # Return empty array for invalid entries
            except (ValueError, SyntaxError):
                return np.array([])  # Handle malformed data gracefully

        training_data[col] = training_data[col].apply(safe_literal_eval)

    # Flatten the training data
    flattened_features = []
    for col in ID_columns:
        arrays = np.stack(training_data[col].values)
        array_df = pd.DataFrame(arrays, columns=[f"{col}_{i}" for i in range(arrays.shape[1])])
        flattened_features.append(array_df)

    data_flattened = pd.concat([training_data.drop(ID_columns, axis=1)] + flattened_features, axis=1)

    X = data_flattened.drop(["Exercise Name", "Difficulty (1-5)", "score"], axis=1)
    y = data_flattened["score"]

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(X_scaled, y)

    # Process predict_data
    for col in ID_columns:
        predict_data[col] = predict_data[col].apply(safe_literal_eval)

    # Flatten the test data
    flattened_features_test = []
    for col in ID_columns:
        arrays_test = np.stack(predict_data[col].values)
        array_df_test = pd.DataFrame(arrays_test, columns=[f"{col}_{i}" for i in range(arrays_test.shape[1])])
        flattened_features_test.append(array_df_test)

    data_test_flattened = pd.concat([predict_data.drop(ID_columns, axis=1)] + flattened_features_test, axis=1)

    X_test = data_test_flattened.drop(["Exercise Name", "Difficulty (1-5)"], axis=1)
    X_test_scaled = scaler.transform(X_test)

    y_pred_test = knn.predict(X_test_scaled)

    # Adjust scores using the function
    exercise_difficulty = data_test_flattened["Difficulty (1-5)"]
    adjusted_scores_test = adjust_score(y_pred_test, user_fitness_level, exercise_difficulty)

    data_test_flattened["Predicted Score"] = adjusted_scores_test

    sorted_results = data_test_flattened.sort_values(by="Predicted Score", ascending=False)
    sorted_results = sorted_results[~sorted_results['Exercise Name'].isin(The_Exercises[i])]
    return sorted_results["Exercise Name"].iloc[0]


In [244]:
"""
Clusters exercises based on Main muscles and Target muscles. First it selects relevant columns for clustering and converts this categorical data to numeric values using OneHotEncoder.
K-Means algrotihm is used to create 10 clusters, and the exercises are grouped and stored in a dictionary, where each cluster contains a list of exercise names.
"""

clustered_features = exercise_data[['Main_muscle', 'Target_Muscles']]

encoder = OneHotEncoder(sparse_output=False)
encoded_features = encoder.fit_transform(clustered_features)

kmeans = KMeans(n_clusters=10, random_state=42)
exercise_data['Cluster'] = kmeans.fit_predict(encoded_features)

clustered_exercises = exercise_data.groupby('Cluster')['Exercise Name'].apply(list).to_dict()

In [ ]:
def select_MainMuscles():
    """
    The user has the opportunity to select one or more Main muscles from a list generated from the exercise data.
    The user select Main muscles by entering the number associated with each Main muscle and can quit with 'q'. 
    
    Returns a set with the selected Main muscles.
    """
    available_muscles = exercise_data['Main_muscle'].unique().tolist()
    selected_muscles = set()

    selecting = True
    while selecting:
        print("Select one or more main muscles from the list:")
        for index, muscle in enumerate(available_muscles, start=1):
            print(f"{index}. {muscle}")

        choice = input("Enter the number of the muscle you want to select (or 'q' to finish selection): ")

        if choice.lower() == 'q':
            selecting = False
        
        else:
            try:
                choice_index = int(choice) - 1
                if 0 <= choice_index < len(available_muscles):
                    selected_muscle = available_muscles[choice_index]
                    if selected_muscle in selected_muscles:
                        print(f"You have already selected: {selected_muscle}. Please choose another muscle.")
                    else:
                        print(f"You have selected: {selected_muscle}")
                        selected_muscles.add(selected_muscle)
                else:
                    print("Invalid choice, please try again.")
            except ValueError:
                print("Please enter a valid number.")

    return selected_muscles

def filter_TargetMuscles(selected_muscles):
    """
    Filters the Target muscles based on the selected Main muscles. For each Main muscle in selected_muscles set, the function looks up the corresponding Target muscles in the exercise data.
    Duplicates are removed.
    
    Returns a list of unique Target muscles.
    """
    target_muscles = set()

    for muscle in selected_muscles:
        filtered_data = exercise_data[exercise_data['Main_muscle'] == muscle]
        target_muscles.update(filtered_data['Target_Muscles'])

    return list(target_muscles)

def PREplanned_workout():
    """
    The user has the oppertunity to select a Pre-planned workout from a list of options:
    - Push
    - Pull
    - Leg
    - Whole Body
    
    Returns a list of Target muscles for the selected workout type.
    """
    print("Select a Pre-planned workout type:")
    print("1. Push")
    print("2. Pull")
    print("3. Leg")
    print("4. Whole Body")

    while True:
        choice = input("Enter the number of the workout type you want to select: ")
        try:
            choice_index = int(choice)
            if choice_index == 1:  # Push
                main_muscles = ['Chest', 'Shoulder', 'Triceps']
                print("You have selected: Push")
                return main_muscles
            elif choice_index == 2:  # Pull
                main_muscles = ['Back', 'Biceps', 'Forearm']
                print("You have selected: Pull")
                return main_muscles
            elif choice_index == 3:  # Leg
                main_muscles = ['Quads', 'Hamstring', 'Glutes', 'Calves']
                print("You have selected: Leg")
                return main_muscles
            elif choice_index == 4:  # Whole Body
                main_muscles = ['Chest', 'Back', 'Shoulder', 'Biceps', 'Triceps', 'Quads', 'Hamstring', 'Glutes', 'Calves']
                print("You have selected: Whole Body")
                return main_muscles
            else:
                print("Invalid choice, please try again.")
        except ValueError:
            print("Please enter a valid number.")

def select_difficulty():
    """
    The user needs to select the difficulty level depending on their experince for the workout.
    The user can choose between levels 1 to 5, with 1 being the easiest and 5 being the hardest.
     
    Returns an integer, the selected difficulty level (1-5).
    """
    print("Select difficulty level")
    while True:
        difficulty = input("Select difficulty level (1-5): ")
        try:
            difficulty = int(difficulty)
            if 1 <= difficulty <= 5:
                print(f"You have selected difficulty level: {difficulty}")
                return difficulty  # Return the selected difficulty level
            else:
                print("Please enter a number between 1 and 5.")
        except ValueError:
            print("Please enter a valid number.")

weekly_workouts = {}

days_of_week = ["MONDAY", "TUESDAY", "WEDNESDAY", "THURSDAY", "FRIDAY", "SATURDAY", "SUNDAY"]

difficulty = select_difficulty()

for day in days_of_week:
    print(f"{day}:")
    print("Select workout type:")
    print("1. Pre-planned")
    print("2. Hybrid")
    print("3. Rest Day")

    valid = False
    while not valid:
        workout = input("Enter the number of the workout type you want to select: ")
        try:
            workout_index = int(workout)
            if workout_index == 1:  # Pre-planned
                main_muscles = PREplanned_workout()
                valid = True

            elif workout_index == 2:  # Hybrid
                main_muscles = select_MainMuscles()
                valid = True

            elif workout_index == 3:  # Rest Day
                print("Rest day selected. No workout planned.")
                weekly_workouts[day] = []
                valid = True

            else:
                print("Invalid choice, please try again.")
        except ValueError:        
            print("Please enter a valid number.")

    if workout_index != 3:
        selected_targets = filter_TargetMuscles(main_muscles)

        if selected_targets:
            weekly_workouts[day] = selected_targets
    
    clear_output()


In [ ]:
# Print the weekly workouts
print("Weekly Workouts:")
for day, exercises in weekly_workouts.items():
    muscles = {muscle.strip() for exercise in exercises for muscle in exercise.split(',') if muscle.strip() != ""}
    weekly_workouts[day] = list(muscles)
print(weekly_workouts)
print(f"Difficulty level for the week is set to: {difficulty}")

In [ ]:
The_Exercises = {"MONDAY" :[], 
"TUESDAY": [], 
"WEDNESDAY": [], 
"THURSDAY": [], 
"FRIDAY": [], 
"SATURDAY": [], 
"SUNDAY": []}
for i in weekly_workouts:
    worked_muscles = []
    for k in weekly_workouts[i]:
        if worked_muscles.count(k) <= 2:
            k = k.replace(",", "") 
            predic_data = encoded_data(k)  
            exercise = knn(predic_data, difficulty, The_Exercises) 
            row = exercise_data[exercise_data['Exercise Name'] == exercise]  
            primary_muscles = row['Target_Muscles'].str.split(',').iloc[0] 
            worked_muscles.extend([muscle.strip() for muscle in primary_muscles if muscle.strip() != ""])
            secondary_muscles = row['Secondary Muscles'].str.split(',').iloc[0] 
            worked_muscles.extend([muscle.strip() for muscle in secondary_muscles if muscle.strip() != ""])
            The_Exercises[i].append(exercise)
print(The_Exercises)



In [248]:
def get_new_exercise(exercise_name, clustered_exercises, used_exercises):
    """
    Returns a new exercise from the same cluster as exercise_name.
    """
    for exercises in clustered_exercises.values():
        if exercise_name in exercises:
            available_exercises = []
            for ex in exercises:
                if ex != exercise_name and ex not in used_exercises:
                    available_exercises.append(ex)
            
            if len(available_exercises) > 0:
                return random.choice(available_exercises)
    
    return exercise_name

def adjust_scheme(all_exercises, clustered_exercises):
    """
    Adjusts the training scheme to avoid repeated exercises between days and within the same day.
    If duplicates occur, they are replaced with a new exercise from the same cluster, with the get_new_exercise function.
    
    Returns a dictionary with the adjusted traning scheme, whit uniquie exercises for each day.
    """
    days = list(all_exercises.keys())
    num_days = len(days)

    used_exercises_all_days = set()

    for day in range(num_days):  # Gå igenom alla dagar
        if day > 0:
            previous_day = days[day - 1]
        else:
            previous_day = None

        current_day = days[day]

        if day < num_days - 1:
            next_day = days[day + 1]
        else:
            next_day = None

        # Hämta övningar från föregående och nästa dag om de finns
        if previous_day:
            used_exercises = set(all_exercises[previous_day])
        else:
            used_exercises = set()

        if next_day:
            used_next_day = set(all_exercises[next_day])
        else:
            used_next_day = set()

        # Combine previous and next day exercises for checking duplicates
        check_exercises = used_exercises.union(used_next_day)

        unique_exercises = set()  # Hålla koll på övningar inom samma dag
        adjusted_exercises = []

        for exercise in all_exercises[current_day]:
            # Förhindra dubletter inom samma dag eller från föregående/nästa dag
            if exercise in unique_exercises or exercise in check_exercises:
                # Byt ut övningen mot en ny från samma kluster
                new_exercise = get_new_exercise(exercise, clustered_exercises, used_exercises_all_days)
                while new_exercise in unique_exercises or new_exercise in check_exercises:
                    new_exercise = get_new_exercise(new_exercise, clustered_exercises, used_exercises_all_days)
                adjusted_exercises.append(new_exercise)
                unique_exercises.add(new_exercise)
            else:
                adjusted_exercises.append(exercise)
                unique_exercises.add(exercise)

        # Uppdatera övningar för dagen med unika övningar
        all_exercises[current_day] = adjusted_exercises

        # Uppdatera den gemensamma listan av använda övningar under veckan
        used_exercises_all_days.update(adjusted_exercises)

    return all_exercises

# Justera schemat för att undvika upprepade övningar
The_Exercises = adjust_scheme(The_Exercises, clustered_exercises)


In [ ]:
data = []

for day, exercises in The_Exercises.items():
    data.append({'Day': day, 'Exercise': '', 'Equipment': '', 'Preparation': '', 'Execution': ''})  # Blank row for the day name
    for exercise in exercises:
        information = exercise_data[exercise_data['Exercise Name'] == exercise]
        data.append({
            'Day': '',  # Leave the day column empty for subsequent exercises
            'Exercise': exercise,
            'Main Muscle': information['Main_muscle'].iloc[0],
            'Equipment': information['Equipment'].iloc[0],
            'Preparation': information['Preparation'].iloc[0],
            'Execution': information['Execution'].iloc[0]
        })
    data.append({'Day': '', 'Exercise': '', 'Main Muscle': '', 'Equipment': '', 'Preparation': '', 'Execution': ''})

# Convert the list to a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
df.head(70)

In [ ]:
output_file = 'final_scheme.xlsx'
df.to_excel(output_file, index=False)